# Purpose

The purpose of this script is to analyze stock following principals outlines by Benjamin Graham. These principles involve the following guidlines

- PE ratio below 9 but positivie
- Current Ratio (Current Assets/Current Liab) greater than 1.5
- Debt Load (Total Debt/ Current Assets) Less than or equal to 1.1
- Positive EPS for each quarter in the last year
- Price To Book ratio less than or equal to 1.2 
- Look for dividends

Additionally I will be calculating the following variables for my own interest
- NCAV (Net Current Assets Value) = (Net Assets-TotalLiabilities-preferred stock)

## Load Name of Stocks
The section uses the yahoo finance library to get the names of many stocks. 

In [1]:
import yahoo_fin.stock_info as si
dow_list = si.tickers_dow() #Loads Tickers in dow jones
nasdaq_list = si.tickers_nasdaq() #Loads Tickers in NASDAQ
sp500_list = si.tickers_sp500()
other_list = si.tickers_other() #Loads Tickers in Other Indexes


## Load yfinance and Create Functions To Read Data I am interested In 
The yfinance library is what will primarily be used to load information for each of the stock tickers. Within this libary 3 primary functions will be called they are   

- stk.info #(used for PE, EPS, Price to Book values)
- stk.balance_sheet (used for Current Ratio, Debt Load, NCAV)
- stk.dividends (used for dividend calculations

In order to use these functions more efficiently they will be integrated into a number of my own creation. The first of these functions is below and is used to read useful information from the stk.info call

In [35]:
import yfinance as yf
import numpy

def extractInfo(Ticker):
    commonStock = False
    divRate = False
    forPE = 0
    trailPE = 0
    forEPS = 0
    trailEPS = 0
    P2B = 0
    
    #Ticers with these values in their name were often for funds preferred stock etc
    #So this line removes those other investment mediums
    if '$' in Ticker or '.' in Ticker:
        return commonStock,divRate,forPE,trailPE,forEPS,trailEPS,P2B
    
    stk = yf.Ticker(Ticker) #Creates object to load stock data
    stkInfo = stk.info #Loads basic stock info 
    
    #In yfinance all common stock have forwardPE listed but funds and other investments do not.
    #So this line removes those other investment mediums
    if 'forwardPE' not in stkInfo or stkInfo['forwardPE'] == None: 
        return commonStock,divRate,forPE,trailPE,forEPS,trailEPS,P2B
    
    commonStock = True
    
    #Reads stock info where it is available to be read
    divRate = stkInfo['dividendRate'] if 'dividendRate' in stkInfo else 'None'
    forPE = stkInfo['forwardPE']
    trailPE = stkInfo['trailingPE'] if 'trailingPE' in stkInfo else 'Negative'
    forEPS = stkInfo['forwardEps'] if 'forwardEps' in stkInfo else 'None'
    trailEPS = stkInfo['trailingEps'] if 'trailingEps' in stkInfo else 'None'
    P2B = stkInfo['priceToBook'] if 'priceToBook' in stkInfo else 'None'
    
    return commonStock,divRate,forPE,trailPE,forEPS,trailEPS,P2B,stk

The second function will be responsible for reading the balance sheet for the stk. It is assumed here that the function extractInfo has already been read ahead of this function and has prevented any non-common-stock investments from progressing further into the data extraction process 

In [67]:
def extractBalSheet(stk):
    stkBal= stk.balancesheet #Returns last four years of annual balance sheet
    recBal = stkBal[stkBal.columns[0]] #Returns the most recent balance sheet
    
    curAsts = recBal['Total Current Assets'] if 'Total Current Assets' in recBal else 'None'
    curLiabs = recBal['Total Current Liabilities'] if 'Total Current Liabilities' in recBal else 'None'
    totAsts = recBal['Total Assets'] if 'Total Assets' in recBal else 'None'
    totLiabs = recBal['Total Liab'] if 'Total Liab' in recBal else 'None'
    
    stkHoldEqty = totAsts-totLiabs if (type(totAsts)!=str and type(totLiabs)!=str) else 'None'
    curRat = curAsts/curLiabs if (type(curAsts)!=str and type(curLiabs)!=str and curLiabs!=0) else 'None'
    debtLoad = totLiabs/curAsts if (type(curAsts)!=str and type(totLiabs)!=str and curAsts!=0) else 'None'
    
    return stkHoldEqty,curRat,debtLoad,curAsts,curLiabs,totAsts,totLiabs
    

The third function is responsible for reading the earnings of the data for the last four monthes.

In [68]:
def extractPrev4QuartEarnings(stk):
    stkQuartEarn = stk.quarterly_earnings
    earn = stkQuartEarn['Earnings'] if 'Earnings' in stkQuartEarn.columns else 'None'
    numQ = earn.shape[0]
    mostRec = earn[-1] if numQ >= 1 else 'None' #Most recent is last value
    secMostRec = earn[-2] if numQ >= 2 else 'None'
    thrdMostRec = earn[-3] if numQ >= 3 else 'None'
    frthMostRec = earn[-4] if numQ >= 4 else 'None'
    
    return mostRec, secMostRec, thrdMostRec, frthMostRec
    

## Loop Through Stocks in the Dow

In [69]:


for Ticker in dow_list:
    commonStock,divRate,forPE,trailPE,forEPS,trailEPS,P2B,stk = extractInfo(Ticker)
    
    if commonStock == True:
        stkHoldEqty,curRat,debtLoad,curAsts,curLiabs,totAsts,totLiabs = extractBalSheet(stk)
        mostRec, secMostRec, thrdMostRec, frthMostRec = extractPrev4QuartEarnings(stk)

4
AAPL
4
AMGN
4
AXP
4
BA
4
CAT
4
CRM
4
CSCO


KeyboardInterrupt: 